# LSTM autoencoder를 이용한 시계열 data 이상 감지

시계열 자료의 이상치 검출

## Load the data

비정상적인 동작 기간이 labeling 된  인공 시계열 데이터
[Numenta Anomaly Benchmark(NAB)](
https://www.kaggle.com/boltzmannbrain/nab) 를 사용합니다. 데이터는 정열되고 타임스탬프가 지정된 단일 값 메트릭입니다.

학습에는 noise가 포함된 정상 시계열 데이터인인 `art_daily_small_noise.csv` 파일을 사용하고 테스트에는 이상치가 포함된 `art_daily_jumpsup.csv` 파일을 사용합니다. 이 데이터 세트의 단순성 덕분에 이상 탐지를 효과적으로 시연할 수 있습니다.

In [ ]:
# 데이터셋의 기본 URL을 설정합니다.
# 정상 데이터 URL 접미사를 설정합니다.
# 정상 데이터의 전체 URL을 생성합니다.
# 정상 데이터(훈련 세트)를 불러옵니다.
# parse_dates=True를 사용하여 'timestamp' 열을 날짜 형식으로 파싱합니다.
# index_col="timestamp"를 사용하여 'timestamp' 열을 인덱스로 설정합니다.
# 비정상 데이터 URL 접미사를 설정합니다.
# 비정상 데이터의 전체 URL을 생성합니다.
# 비정상 데이터(테스트 세트)를 불러옵니다.
# parse_dates=True를 사용하여 'timestamp' 열을 날짜 형식으로 파싱합니다.
# index_col="timestamp"를 사용하여 'timestamp' 열을 인덱스로 설정합니다.
# 정상 데이터와 비정상 데이터의 형태(shape)를 출력합니다.

## data 시각화
### 정상 시계열 데이터

- 정상 데이터를 autoencoder 훈련에 사용합니다.

### 이상이 있는 시계열 데이터

정상 데이터로 훈련된 autoencoder 모델의 테스트를 위해 다음 데이터를 사용하고, 데이터의 갑작스러운 점프가 이상으로 감지되는지 확인합니다.

## 훈련 데이터 준비

훈련 시계열 데이터 파일(`df_small_noise`)에서 데이터 값을 가져오고 `value` 데이터를 정규화 합니다. 14일 동안 5분마다 `value`가 있습니다.

-   24시간 * 60분 / 5분 = **288 timesteps per day**
-   288 timesteps per day * 14일 = **total 4032 data points**

In [ ]:
# 테스트 데이터를 정규화하기 위해 평균과 표준 편차를 정규화하고 저장합니다.
# standard scaling

In [ ]:
# normalize된 df_small_noise

### 시퀀스 data 생성

훈련 데이터에서 `TIME_STEPS` 연속 데이터 값을 조합하여 시퀀스를 생성합니다.

In [ ]:
# 모델에서 사용하기 위해 생성된 훈련 시퀀스
def create_sequences(values, time_steps=TIME_STEPS):

## LSTM Autoencoder model


<img src="https://www.researchgate.net/profile/Hoang_Duy_Trinh2/publication/336594630/figure/fig2/AS:814792885420033@1571273170513/LSTM-Autoencoder-for-Anomaly-Detection.png" width=400 />

- RepeatVector 이해  

<img src="https://i.stack.imgur.com/LNXjF.jpg" width=400 />

| 구분               | return\_sequences=True          | RepeatVector                  |
| ---------------- | ------------------------------- | ----------------------------- |
| 역할               | LSTM이 모든 타임스텝의 출력을 반환           | 하나의 벡터를 여러 타임스텝으로 반복          |
| Encoder에서 주로 사용? | ❌ (보통 False로 해서 마지막 state만 씀)   | ✅ (잠재차원 z를 Decoder 입력 형태로 준비)             |
| 출력 형태            | (batch, timesteps, latent\_dim) | (batch, timesteps, latent\_dim)       |
| 목적               | Encoder가 "시퀀스 형태"의 특성을 유지할 때    | Decoder가 "시퀀스 형태" 입력을 필요로 할 때 |


In [ ]:
# 시퀀셜 모델을 생성합니다.
# 첫 번째 LSTM 레이어를 추가합니다.
# 유닛 수는 32, 입력 형태는 (시퀀스 길이, 특성 수)로 설정합니다.
# 드롭아웃 레이어를 추가하여 과적합을 방지합니다.
# RepeatVector 레이어를 추가하여 인코더의 출력 길이를 시퀀스 길이로 반복합니다.
# 드롭아웃 레이어를 추가하여 과적합을 방지합니다.
# 두 번째 LSTM 레이어를 추가합니다.
# 유닛 수는 32, 모든 시퀀스를 출력하도록 설정(return_sequences=True)합니다.
# Dense 레이어를 추가하여 출력 형태를 (시퀀스 길이, 특성 수)로 맞춥니다.
# 원하는 learning rate를 설정합니다.

## 모델 train

재구성 모델이기 때문에 Input과 Target 모두 `X_train`을 사용합니다.

In [ ]:
# 1. 시계열 데이터 분할
# 2. 모델 학습 (shuffle=False로 설정)

## 이상 감지

모델이 입력 데이터를 얼마나 잘 재구성할 수 있는지 결정하여 이상을 감지합니다.

1.   훈련 샘플에서 MAE 손실을 찾습니다.
2.   최대 MAE 손실 값을 찾습니다. 이것은 우리 모델이 샘플을 재구성하기 위해 수행한 최악의 작업입니다. 이를 이상 감지의 `threshold`로 설정합니다.
3.   샘플의 재구성 손실이 이 `threshold` 보다 크면 모델이 익숙하지 않은 패턴을 보고 있다고 추론할 수 있습니다. 우리는 이 샘플을 `anomaly`로 분류할 것입니다.

- MAE 손실 시각화

In [ ]:
# 모델을 사용하여 훈련 데이터에 대한 예측값을 생성합니다.
# 훈련 데이터와 예측값 사이의 MAE(Mean Absolute Error) 손실을 계산합니다.
# np.abs(X_train_pred - X_train)은 절대 오차를 계산합니다.
# np.mean(..., axis=1)은 각 샘플에 대해 평균 절대 오차를 계산합니다.
# 훈련 데이터의 MAE 손실을 히스토그램으로 시각화합니다.

- train set의 최대 MAE 손실을 재구성 손실 임계값으로 정합니다.

### 재구성 비교

우리 모델이 첫 번째 샘플을 어떻게 재구성했는지 봅니다.
이것은 훈련 데이터 세트 1일차의 288개 timestep 입니다.

In [ ]:
# 첫 번째 시퀀스가  어떻게 학습되었는지 확인

### 테스트 데이터 준비
- Test set을 standard scale   
- 분포가 바뀌면 안되므로 Train set 에서 구해 놓았던 평균과 표준 편차를 그대로 사용
- 시각화

In [ ]:
# 테스트 데이터(df_daily_jumpsup)를 표준 정규분포로 변환
# 그래프를 그리기 위한 Figure와 Axes 객체를 생성합니다.
# 표준화된 테스트 데이터(df_test_value)를 플롯합니다.
# legend=False는 범례를 표시하지 않도록 설정합니다.

- Test set에서 시퀀스 data 생성

- test set의 MAE loss 구하기

In [ ]:
# 모델을 사용하여 테스트 데이터에 대한 예측값을 생성합니다.
# 테스트 데이터와 예측값 사이의 MAE(Mean Absolute Error) 손실을 계산합니다.
# np.abs(x_test_pred - x_test)은 절대 오차를 계산합니다.
# np.mean(..., axis=1)은 각 샘플에 대해 평균 절대 오차를 계산합니다.
# 테스트 MAE 손실을 1차원 배열로 변환합니다.

- MAE 손실 시각화

In [ ]:
# 테스트 데이터의 MAE 손실을 히스토그램으로 시각화합니다.
# threshold 값을 수직선으로 표시합니다.

- 모든 이상 샘플 감지

## 이상치 plot

이제 우리는 비정상 데이터 샘플을 알고 있습니다. 이를 통해 원본 테스트 데이터에서 해당 `timestamps`를 찾을 수 있습니다. 이를 위해 다음 방법을 사용할 것입니다.

time_steps = 3이고 10개의 훈련 값이 있다고 가정해 보겠습니다. 우리의 `X_train`은 다음과 같이 보일 것입니다:

- 0, 1, 2
- 1, 2, 3
- 2, 3, 4
- 3, 4, 5
- 4, 5, 6
- 5, 6, 7
- 6, 7, 8
- 7, 8, 9

초기 및 마지막 time_steps-1 데이터 값을 제외한 모든 값은 `time_steps` 샘플 수로 표시됩니다. 따라서 샘플 [(3, 4, 5), (4, 5, 6), (5, 6, 7)]이 비정상이라는 것을 안다면 데이터 포인트 5가 비정상이라고 말할 수 있습니다.  

그러므로 `TIME_STEPS`기간 전체 샘플 `[(i - timesteps + 1) ~ (i)]`이 비정상인 경우 데이터 `i`는 비정상으로 분류할 수 있습니다.

In [ ]:
# 비정상 데이터의 시작 인덱스를 설정합니다.
# 지정된 인덱스 범위 내의 모든 데이터가 비정상인지 확인합니다.
# anomalies 배열의 'index'부터 'index + TIME_STEPS'까지의 모든 값이 True인지 검사합니다.
# True이면 해당 범위의 모든 데이터가 비정상임을 의미합니다.
# 결과를 출력합니다. True이면 해당 범위의 모든 데이터가 비정상임을 의미합니다.

In [ ]:
# 이상 데이터의 인덱스를 저장할 리스트를 초기화합니다.
# 데이터의 인덱스를 순회합니다.
    # 현재 인덱스에서 TIME_STEPS만큼 이전까지의 모든 값이 이상인지 확인합니다.
        # 모든 값이 이상인 경우, 현재 인덱스를 이상 데이터 인덱스 리스트에 추가합니다.
# 이상 데이터 인덱스를 출력합니다.

원본 테스트 데이터 플롯에 이상치를 오버레이해 시각화

In [ ]:
# 이상 데이터의 인덱스를 사용하여 해당하는 부분 데이터프레임을 선택합니다.
# 그래프를 그리기 위한 Figure와 Axes 객체를 생성합니다.
# 전체 데이터(df_daily_jumpsup)를 플롯합니다.
# legend=False는 범례를 표시하지 않도록 설정합니다.
# 이상 데이터(df_subset)를 플롯합니다.
# legend=False는 범례를 표시하지 않도록 설정합니다.
# color="r"는 이상 데이터를 빨간색으로 표시합니다.